<a href="https://colab.research.google.com/github/vasiliyeskin/MachineLearningExperiences/blob/master/cifar10/CIFAR10_for_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Свёрточная нейронная сеть CIFAR-10**

In [2]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

In [3]:
# Задаем seed для повторяемости результатов
numpy.random.seed(42)

In [4]:
# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 10
# Количество эпох для обучения
nb_epoch = 25
# Размер изображений
img_rows, img_cols = 32, 32
# Количество каналов в изображении: RGB
img_channels = 3

170500096/170498071 [==============================] - 2s 0us/step


In [6]:
# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255 

In [7]:
# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## Создание модели

In [8]:
# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой: 32 карты признаков, ядро свёртки 3x3, размер входных данных 32x32 в 3 канала
model.add(Conv2D(filters=32, (3, 3), padding='valid',
                        input_shape=(32, 32, 3), activation='relu',data_format="channels_last"))
# Второй сверточный слой
model.add(Conv2D(filters=32, (3, 3), activation='relu', padding='valid',data_format="channels_last"))
# Первый слой подвыборки: размер уменьшения слоя 2x2 с выбором максимального значения из этого квадрата
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last"))
# Слой регуляризации Dropout: при обучении случайным образом выключается 25% неронов - необходимо для предотвращения переобучения
model.add(Dropout(0.25))

# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='valid', activation='relu',data_format="channels_last"))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu',data_format="channels_last"))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last"))
# Слой регуляризации Dropout
model.add(Dropout(0.25))
# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))
# Слой регуляризации Dropout
model.add(Dropout(0.5))
# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))

In [9]:

# Задаем параметры оптимизации
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [10]:
# Обучаем модель
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,        # перемешивание данных перед каждой эпохой обучения
              verbose=2)

Train on 45000 samples, validate on 5000 samples
Epoch 1/25
 - 224s - loss: 2.3036 - accuracy: 0.0992 - val_loss: 2.3044 - val_accuracy: 0.0970
Epoch 2/25


KeyboardInterrupt: ignored

In [11]:
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

Точность работы на тестовых данных: 10.00%


In [ ]:
print("Сохраняем сеть")
# Сохраняем сеть для последующего использования
# Генерируем описание модели в формате json
model_json = model.to_json()
json_file = open("cifar10_model.json", "w")
# Записываем архитектуру сети в файл
json_file.write(model_json)
json_file.close()
# Записываем данные о весах в файл
model.save_weights("cifar10_model.h5")
print("Сохранение сети завершено")